In [2]:
import cv2
import numpy as np
from collections import deque # deque is used for fast pop and append with oi time complexity comapare to list with on time


In [11]:
yellowlower=np.array([0,91,96])
yellowupper=np.array([38,255,235])
kernel =np.ones((5,5),np.uint8)

In [13]:
bpoints=[deque(maxlen=512)]
gpoints=[deque(maxlen=512)]
rpoints=[deque(maxlen=512)]
ypoints=[deque(maxlen=512)]

bindex=0
gindex=0
rindex=0
yindex=0

colors=[(255,0,0),(0,255,0),(0,0,255),(0,255,255)]
colorindex=0

frame1 = np.zeros([480,640,3]) + 255

frame1 = cv2.rectangle(frame1, (40,1), (140,65), (0,0,0), 2)
frame1 = cv2.rectangle(frame1, (160,1), (255,65), colors[0], -1)
frame1 = cv2.rectangle(frame1, (275,1), (370,65), colors[1], -1)
frame1 = cv2.rectangle(frame1, (390,1), (485,65), colors[2], -1)
frame1 = cv2.rectangle(frame1, (505,1), (600,65), colors[3], -1)
# frame1 = cv2.rectangle(frame1, (620,1), (720,65), (0,0,0), 2)


cv2.putText(frame1, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(frame1, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(frame1, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(frame1, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(frame1, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
# cv2.putText(frame1, "SAVE", (650, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)
camera = cv2.VideoCapture(0)
# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Check to see if we have reached the end of the video (useful when input is a video file not a live video stream)
   
    
    # Add the same paint interface to the camera feed captured through the webcam (for ease of usage)
    frame = cv2.rectangle(frame, (30,65), (80,125), (0,0,0), 2)
    frame = cv2.rectangle(frame, (100,65), (150,125), colors[0], -1)
    frame = cv2.rectangle(frame, (170,65), (220,125), colors[1], -1)
    frame = cv2.rectangle(frame, (240,65), (290,125), colors[2], -1)
    frame = cv2.rectangle(frame, (310,65), (360,125), colors[3], -1)
#     frame = cv2.rectangle(frame, (620,1), (720,65), (0,0,0), 2)


    cv2.putText(frame, "CLEAR ALL", (39, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (110, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (180, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (250, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (320, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
#     cv2.putText(frame, "SAVE", (650, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
    
    if not grabbed:
        break 
        # Determine which pixels fall within the yellow boundaries and then blur the binary image
    yellowMask = cv2.inRange(hsv, yellowlower, yellowupper)
    yellowMask = cv2.erode(yellowMask, kernel, iterations=2)
    yellowMask = cv2.morphologyEx(yellowMask, cv2.MORPH_OPEN, kernel)
    yellowMask = cv2.dilate(yellowMask, kernel, iterations=1)

    # Find contours in the image
    ( cnts, _) = cv2.findContours(yellowMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center=None
    if len(cnts)>0:
        cnt=sorted(cnts,key=cv2.contourArea,reverse=True)[0]
        ((x,y),radius)=cv2.minEnclosingCircle(cnt)
        cv2.circle(frame,(int(x),int(y)),int(radius),(255,0,0),2)
        m=cv2.moments(cnt)
        center=(int(m['m10'] / m['m00']),int(m['m01']/m['m00']))
        
        if center[1]<=125:
            if 30<= center[0] <=80:
                bpoints=[deque(maxlen=512)]
                gpoints=[deque(maxlen=512)]
                rpoints=[deque(maxlen=512)]
                ypoints=[deque(maxlen=512)]

                bindex=0
                gindex=0
                rindex=0
                yindex=0
                frame1[67:,:,:]=255#cgange here
            elif 100<= center[0]<=150:
                colorindex=0
            elif 170<= center[0]<=220:
                colorindex=1
            elif 240<= center[0]<=290:
                colorindex=2
            elif 310<= center[0]<=360:
                colorindex=3
        else:
            if colorindex==0:
                bpoints[bindex].appendleft(center)
            elif colorindex==1:
                gpoints[bindex].appendleft(center)   
            elif colorindex==2:
                rpoints[bindex].appendleft(center)
            elif colorindex==3:
                ypoints[bindex].appendleft(center)
    else:
        bpoints.append(deque(maxlen=512))
        bindex+=1
        gpoints.append(deque(maxlen=512))
        gindex+=1
        rpoints.append(deque(maxlen=512))
        rindex+=1
        ypoints.append(deque(maxlen=512))
        yindex+=1
        
    points=[bpoints,gpoints,rpoints,ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1,len(points[i][j])):
                if points[i][j][k-1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame,points[i][j][k-1],points[i][j][k],colors[i],2)
                cv2.line(frame1,points[i][j][k-1],points[i][j][k],colors[i],2)
    cv2.imshow("tracking",frame)
#     cv2.imshow("Paint",frame1)
    
    if cv2.waitKey(1) &0xFF ==ord('q'):
        break
camera.release()
cv2.destroyAllWindows()
    

In [8]:
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

[deque([])]

In [6]:
frame.shape

(480, 640, 3)